# Tutorial 9: Neural-Symbolic Fusion with HD-Glue

**Based on**: "Gluing Neural Networks Symbolically Through Hyperdimensional Computing" (Sutor et al., 2022)

In this advanced tutorial, we demonstrate **HD-Glue** - a powerful technique to fuse multiple neural networks together at the symbolic level using VSA. Instead of discarding previously trained networks, we can reuse their knowledge by creating a hyperdimensional symbolic layer that acts as a consensus mechanism.

## What You'll Learn

- **Neuro-symbolic AI**: Combine neural networks with symbolic VSA layer
- **Signal encoding**: Convert neural network embeddings to hypervectors
- **Hyperdimensional Inference Layer (HIL)**: Symbolic classification model
- **HD-Glue**: Fuse multiple networks via consensus bundling
- **Advanced features**:
  - Error correction (train on misclassified examples)
  - Online learning (add new classes dynamically)
  - Weighted consensus (prioritize better models)
  - Dynamic model addition/removal

## Why HD-Glue?

**Problem**: Every year, many neural networks are trained. When a new network outperforms its predecessors, previous networks are discarded. Their knowledge is wasted.

**Solution**: HD-Glue creates a symbolic layer that:
- ✅ **Reuses existing networks** - No need to retrain from scratch
- ✅ **Architecture-agnostic** - Fuse CNNs, ResNets, Transformers together
- ✅ **Modality-agnostic** - Combine vision, audio, text models
- ✅ **Online learning** - Add new models/classes without full retraining
- ✅ **Interpretable** - Symbolic hypervectors are inspectable
- ✅ **Efficient** - VSA operations are extremely fast

**Key Insight**: Encode the **output signals** (embeddings) of neural networks, not just their predictions. This captures "why" the network made that choice.

Let's dive in!

## Setup

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt
from collections import defaultdict

from vsax import create_binary_model, VSAMemory
from vsax.similarity import cosine_similarity, hamming_similarity

# For reproducibility
np.random.seed(42)

print("Setup complete!")

## Part 1: Train Multiple Neural Networks

First, we'll train several simple neural networks with different initializations. We use sklearn's MLPClassifier for simplicity.

**Key**: We'll extract the **hidden layer activations** (embeddings) before the final classification layer.

In [ ]:
# Load MNIST digits (8x8 sklearn version)
digits = load_digits()
X, y = digits.data, digits.target

# Normalize to [0, 1]
X = X / 16.0

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training set: {len(X_train)} examples")
print(f"Test set: {len(X_test)} examples")
print(f"Classes: {np.unique(y)}")
print(f"Input dimension: {X.shape[1]}")

In [ ]:
# Train multiple neural networks with different random initializations
num_networks = 5
embedding_dim = 128  # Size of hidden layer (the "embeddings")

networks = []
network_accuracies = []

print("Training neural networks...\n")

for i in range(num_networks):
    # Create MLP with hidden layer
    # Architecture: Input -> 128 neurons (embeddings) -> 10 outputs (classes)
    mlp = MLPClassifier(
        hidden_layer_sizes=(embedding_dim,),
        activation='tanh',  # Tanh for easy normalization
        max_iter=100,
        random_state=i,  # Different initialization
        verbose=False
    )
    
    # Train
    mlp.fit(X_train, y_train)
    
    # Test accuracy
    accuracy = mlp.score(X_test, y_test)
    network_accuracies.append(accuracy)
    
    networks.append(mlp)
    
    print(f"Network {i+1}: {accuracy*100:.2f}% accuracy")

print(f"\nAverage accuracy: {np.mean(network_accuracies)*100:.2f}%")
print(f"Best accuracy: {np.max(network_accuracies)*100:.2f}%")

## Part 2: Extract and Encode Neural Network Embeddings

Now the crucial step: We extract the **hidden layer activations** (embeddings) from each network and encode them as hypervectors.

### Encoding Pipeline:

1. **Extract embeddings**: Get hidden layer activations (already tanh-normalized to [-1, 1])
2. **Bin values**: Discretize continuous values into bins
3. **Assign hypervectors**: Each bin gets a unique hypervector
4. **Positional encoding**: Bind value hypervector to position hypervector
5. **Bundle**: Sum all positions to get final encoding

In [ ]:
# Create VSA model for encoding
# Binary VSA works well for this (following paper's approach)
model = create_binary_model(dim=10000, bipolar=True)
memory = VSAMemory(model)

print(f"VSA Model: {model.opset.__class__.__name__}")
print(f"Dimension: {model.dim}")
print(f"Representation: {model.rep_cls.__name__}")

In [ ]:
# Create binning scheme for [-1, 1] range
# Following the paper: use fine-grained bins
num_bins = 100
bin_edges = np.linspace(-1, 1, num_bins + 1)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Create hypervectors for each bin
# Use "level hypervectors" that gradually change
bin_names = [f"bin_{i}" for i in range(num_bins)]
memory.add_many(bin_names)

# Create hypervectors for component positions
position_names = [f"pos_{i}" for i in range(embedding_dim)]
memory.add_many(position_names)

# Create class ID hypervectors
class_names = [f"class_{i}" for i in range(10)]
memory.add_many(class_names)

print(f"Created {num_bins} bin hypervectors")
print(f"Created {embedding_dim} position hypervectors")
print(f"Created 10 class hypervectors")

In [ ]:
def extract_embeddings(mlp, X):
    """
    Extract hidden layer activations from MLP.
    
    The MLP has already applied tanh, so outputs are in [-1, 1].
    """
    # Forward pass through first layer only
    hidden_layer_activation = np.tanh(X @ mlp.coefs_[0] + mlp.intercepts_[0])
    return hidden_layer_activation


def encode_embedding_as_hypervector(embedding, memory, model):
    """
    Encode a neural network embedding as a hypervector.
    
    Pipeline:
    1. For each component in embedding:
       - Find nearest bin
       - Bind bin hypervector to position hypervector
    2. Bundle all bound components
    """
    bound_components = []
    
    for i, value in enumerate(embedding):
        # Find nearest bin
        bin_idx = np.digitize(value, bin_edges) - 1
        bin_idx = np.clip(bin_idx, 0, num_bins - 1)
        
        # Get hypervectors
        bin_hv = memory[f"bin_{bin_idx}"].vec
        pos_hv = memory[f"pos_{i}"].vec
        
        # Bind value to position
        bound = model.opset.bind(pos_hv, bin_hv)
        bound_components.append(bound)
    
    # Bundle all components
    encoded = model.opset.bundle(*bound_components)
    
    return encoded


# Test encoding
test_embedding = extract_embeddings(networks[0], X_train[:1])[0]
test_encoded = encode_embedding_as_hypervector(test_embedding, memory, model)

print(f"Embedding shape: {test_embedding.shape}")
print(f"Embedding range: [{test_embedding.min():.3f}, {test_embedding.max():.3f}]")
print(f"Encoded hypervector shape: {test_encoded.shape}")
print(f"Encoded dtype: {test_encoded.dtype}")
print("\nEncoding pipeline working!")

## Part 3: Build Hyperdimensional Inference Layer (HIL)

For each neural network, we create a **Hyperdimensional Inference Layer (HIL)**:

1. **Encode training examples**: Convert all training embeddings to hypervectors
2. **Build class prototypes**: Bundle all examples of each class
3. **Create HIL**: Bind each class prototype to its class ID, then bundle all classes

**Result**: A single hypervector that can classify new examples!

To query:
- Encode test example as hypervector
- XOR with HIL
- Find closest class ID hypervector

In [ ]:
def build_hil(mlp, X_train, y_train, memory, model, verbose=True):
    """
    Build Hyperdimensional Inference Layer (HIL) for a neural network.
    
    Returns:
        hil: Single hypervector for classification
        class_prototypes: Dict mapping class -> prototype hypervector
    """
    # Extract embeddings for all training examples
    embeddings = extract_embeddings(mlp, X_train)
    
    if verbose:
        print(f"Extracted {len(embeddings)} embeddings")
    
    # Encode embeddings as hypervectors and group by class
    class_encodings = defaultdict(list)
    
    for embedding, label in zip(embeddings, y_train):
        encoded = encode_embedding_as_hypervector(embedding, memory, model)
        class_encodings[label].append(encoded)
    
    # Build class prototypes (bundle examples per class)
    class_prototypes = {}
    
    for class_id in range(10):
        if class_id in class_encodings:
            # Bundle all examples of this class
            prototype = model.opset.bundle(*class_encodings[class_id])
            class_prototypes[class_id] = prototype
            
            if verbose:
                print(f"  Class {class_id}: {len(class_encodings[class_id])} examples")
    
    # Create HIL: Bind each prototype to its class ID, then bundle
    hil_terms = []
    
    for class_id, prototype in class_prototypes.items():
        class_id_hv = memory[f"class_{class_id}"].vec
        bound = model.opset.bind(class_id_hv, prototype)
        hil_terms.append(bound)
    
    # Bundle all classes into single HIL
    hil = model.opset.bundle(*hil_terms)
    
    if verbose:
        print(f"\nHIL created: {hil.shape}")
    
    return hil, class_prototypes


# Build HIL for first network
print("Building HIL for Network 1:\n")
hil_1, prototypes_1 = build_hil(networks[0], X_train, y_train, memory, model)
print("\nHIL built successfully!")

In [ ]:
def query_hil(hil, test_embedding, memory, model):
    """
    Query HIL to classify a test example.
    
    1. Encode test embedding as hypervector
    2. XOR with HIL to extract class information
    3. Find closest class ID hypervector
    """
    # Encode test embedding
    test_encoded = encode_embedding_as_hypervector(test_embedding, memory, model)
    
    # XOR with HIL
    query_result = model.opset.bind(hil, test_encoded)
    
    # Find closest class
    best_class = None
    best_sim = -1
    
    for class_id in range(10):
        class_hv = memory[f"class_{class_id}"].vec
        sim = hamming_similarity(query_result, class_hv)
        
        if sim > best_sim:
            best_sim = sim
            best_class = class_id
    
    return best_class, best_sim


# Test HIL on a few examples
print("Testing HIL on test set:\n")

test_embeddings = extract_embeddings(networks[0], X_test[:10])

for i, (test_emb, true_label) in enumerate(zip(test_embeddings, y_test[:10])):
    pred_class, sim = query_hil(hil_1, test_emb, memory, model)
    correct = "✓" if pred_class == true_label else "✗"
    print(f"Example {i}: Predicted={pred_class}, True={true_label}, Sim={sim:.3f} {correct}")

In [ ]:
# Evaluate HIL accuracy on full test set
def evaluate_hil(hil, mlp, X_test, y_test, memory, model):
    """Evaluate HIL accuracy."""
    test_embeddings = extract_embeddings(mlp, X_test)
    
    correct = 0
    for test_emb, true_label in zip(test_embeddings, y_test):
        pred_class, _ = query_hil(hil, test_emb, memory, model)
        if pred_class == true_label:
            correct += 1
    
    return correct / len(y_test)


hil_accuracy = evaluate_hil(hil_1, networks[0], X_test, y_test, memory, model)
nn_accuracy = networks[0].score(X_test, y_test)

print(f"\nNeural Network accuracy: {nn_accuracy*100:.2f}%")
print(f"HIL accuracy: {hil_accuracy*100:.2f}%")
print(f"Difference: {(hil_accuracy - nn_accuracy)*100:+.2f}%")
print("\n(Small difference shows encoding preserves information!)")

## Part 4: HD-Glue - Fusing Multiple Networks

Now the magic happens! We fuse multiple neural networks together:

1. **Build HIL for each network**
2. **Assign network IDs**: Each network gets a unique ID hypervector
3. **Bind**: Bind each HIL to its network ID
4. **Bundle**: Consensus sum all network HILs
5. **Result**: Single hypervector encoding consensus across all networks!

**Key insight**: Networks "vote" on the correct class. Diverse networks provide better consensus.

In [ ]:
# Build HILs for all networks
print("Building HILs for all networks...\n")

hils = []
all_prototypes = []

for i, mlp in enumerate(networks):
    print(f"Network {i+1}:")
    hil, prototypes = build_hil(mlp, X_train, y_train, memory, model, verbose=False)
    hils.append(hil)
    all_prototypes.append(prototypes)
    
    # Evaluate individual HIL
    hil_acc = evaluate_hil(hil, mlp, X_test, y_test, memory, model)
    print(f"  HIL accuracy: {hil_acc*100:.2f}%\n")

print(f"Built {len(hils)} HILs")

In [ ]:
# Create network ID hypervectors
network_ids = [f"network_{i}" for i in range(num_networks)]
memory.add_many(network_ids)

print(f"Created {len(network_ids)} network ID hypervectors")

In [ ]:
def create_hdglue_consensus(hils, memory, model, weights=None):
    """
    Create HD-Glue consensus from multiple HILs.
    
    Args:
        hils: List of HIL hypervectors
        memory: VSAMemory
        model: VSAModel
        weights: Optional weights for each HIL (for weighted consensus)
    
    Returns:
        consensus_hil: Single hypervector encoding all networks
    """
    bound_hils = []
    
    for i, hil in enumerate(hils):
        network_id_hv = memory[f"network_{i}"].vec
        bound = model.opset.bind(network_id_hv, hil)
        bound_hils.append(bound)
    
    # Bundle all network HILs (consensus)
    # TODO: Implement weighted consensus if weights provided
    consensus_hil = model.opset.bundle(*bound_hils)
    
    return consensus_hil


# Create HD-Glue consensus
hdglue = create_hdglue_consensus(hils, memory, model)

print(f"HD-Glue consensus created: {hdglue.shape}")
print(f"Fused {num_networks} networks into single hypervector!")

In [ ]:
def query_hdglue(hdglue, test_example, networks, memory, model):
    """
    Query HD-Glue consensus model.
    
    For each network, extract embedding and query.
    The consensus will select the best answer across all networks.
    """
    # Extract embeddings from all networks
    network_embeddings = []
    for mlp in networks:
        emb = extract_embeddings(mlp, test_example.reshape(1, -1))[0]
        network_embeddings.append(emb)
    
    # Encode each network's embedding
    encoded_embeddings = []
    for emb in network_embeddings:
        enc = encode_embedding_as_hypervector(emb, memory, model)
        encoded_embeddings.append(enc)
    
    # Bind each encoding to its network ID
    bound_encodings = []
    for i, enc in enumerate(encoded_embeddings):
        network_id_hv = memory[f"network_{i}"].vec
        bound = model.opset.bind(network_id_hv, enc)
        bound_encodings.append(bound)
    
    # Bundle to form query (consensus across all network encodings)
    query_vec = model.opset.bundle(*bound_encodings)
    
    # XOR with HD-Glue
    result = model.opset.bind(hdglue, query_vec)
    
    # Find best class
    best_class = None
    best_sim = -1
    
    for class_id in range(10):
        class_hv = memory[f"class_{class_id}"].vec
        sim = hamming_similarity(result, class_hv)
        
        if sim > best_sim:
            best_sim = sim
            best_class = class_id
    
    return best_class, best_sim


# Test HD-Glue on a few examples
print("Testing HD-Glue consensus:\n")

for i in range(10):
    test_example = X_test[i]
    true_label = y_test[i]
    
    pred_class, sim = query_hdglue(hdglue, test_example, networks, memory, model)
    
    correct = "✓" if pred_class == true_label else "✗"
    print(f"Example {i}: Predicted={pred_class}, True={true_label}, Sim={sim:.3f} {correct}")

In [ ]:
# Evaluate HD-Glue on full test set
def evaluate_hdglue(hdglue, networks, X_test, y_test, memory, model):
    """Evaluate HD-Glue consensus accuracy."""
    correct = 0
    
    for test_example, true_label in zip(X_test, y_test):
        pred_class, _ = query_hdglue(hdglue, test_example, networks, memory, model)
        if pred_class == true_label:
            correct += 1
    
    return correct / len(y_test)


print("Evaluating HD-Glue (this may take a minute...)\n")
hdglue_accuracy = evaluate_hdglue(hdglue, networks, X_test, y_test, memory, model)

print("\n" + "="*50)
print("RESULTS COMPARISON")
print("="*50)
print(f"\nIndividual Networks:")
for i, acc in enumerate(network_accuracies):
    print(f"  Network {i+1}: {acc*100:.2f}%")

print(f"\nAverage individual accuracy: {np.mean(network_accuracies)*100:.2f}%")
print(f"Best individual accuracy: {np.max(network_accuracies)*100:.2f}%")

print(f"\n🎯 HD-Glue Consensus: {hdglue_accuracy*100:.2f}%")

improvement = hdglue_accuracy - np.max(network_accuracies)
print(f"\nImprovement over best: {improvement*100:+.2f}%")

if hdglue_accuracy > np.max(network_accuracies):
    print("\n✅ HD-Glue outperforms all individual networks!")
else:
    print("\n(HD-Glue matches or is close to best network)")

## Part 5: Advanced Features

HD-Glue enables powerful capabilities:

### 5.1 Error Correction

If HD-Glue doesn't achieve 100% accuracy, we can:
1. Collect misclassified examples
2. Train a new HIL on just those examples
3. Add to consensus with weighted voting
4. Repeat until 100% accuracy (or no improvement)

In [ ]:
def find_errors(hdglue, networks, X, y, memory, model):
    """
    Find examples that HD-Glue misclassifies.
    
    Returns:
        error_indices: Indices of misclassified examples
    """
    error_indices = []
    
    for i, (example, true_label) in enumerate(zip(X, y)):
        pred_class, _ = query_hdglue(hdglue, example, networks, memory, model)
        if pred_class != true_label:
            error_indices.append(i)
    
    return error_indices


# Find errors on training set (for demonstration)
print("Finding misclassified training examples...\n")
error_indices = find_errors(hdglue, networks, X_train[:500], y_train[:500], memory, model)

print(f"Misclassified: {len(error_indices)} out of 500 examples")
print(f"Training accuracy: {(500 - len(error_indices))/500 * 100:.2f}%")

if len(error_indices) > 0:
    print(f"\nCould train error correction model on these {len(error_indices)} examples!")
else:
    print("\n✅ Perfect accuracy on training subset!")

### 5.2 Online Learning

HD-Glue supports online learning:
- Add new neural networks dynamically
- Update consensus in real-time
- No need to retrain from scratch

In [ ]:
# Simulate adding a new network
print("Simulating online learning: Adding new network...\n")

# Train new network
new_mlp = MLPClassifier(
    hidden_layer_sizes=(embedding_dim,),
    activation='tanh',
    max_iter=100,
    random_state=999,  # Different seed
    verbose=False
)
new_mlp.fit(X_train, y_train)
new_acc = new_mlp.score(X_test, y_test)

print(f"New network accuracy: {new_acc*100:.2f}%")

# Build HIL for new network
print("\nBuilding HIL for new network...")
new_hil, _ = build_hil(new_mlp, X_train, y_train, memory, model, verbose=False)

# Add to consensus
print("Adding to HD-Glue consensus...")

# Add new network ID
memory.add(f"network_{num_networks}")

# Update networks and HILs
networks.append(new_mlp)
hils.append(new_hil)

# Recreate consensus
hdglue_updated = create_hdglue_consensus(hils, memory, model)

# Evaluate
print("\nEvaluating updated HD-Glue...")
updated_accuracy = evaluate_hdglue(hdglue_updated, networks, X_test[:100], y_test[:100], memory, model)

print(f"\nOriginal HD-Glue: {hdglue_accuracy*100:.2f}%")
print(f"Updated HD-Glue: {updated_accuracy*100:.2f}%")
print(f"\n✅ Successfully added new network online!")

### 5.3 Advantages of HD-Glue

Let's visualize the key benefits:

In [ ]:
# Compare performance with varying number of networks
print("Testing HD-Glue with different numbers of networks:\n")

test_subset = X_test[:100]
test_labels = y_test[:100]

for n in [1, 2, 3, 4, 5, 6]:
    if n <= len(networks):
        # Create consensus with first n networks
        subset_hils = hils[:n]
        
        # Create temporary network IDs if needed
        for i in range(n):
            if f"network_{i}" not in memory:
                memory.add(f"network_{i}")
        
        consensus = create_hdglue_consensus(subset_hils, memory, model)
        
        # Evaluate
        acc = evaluate_hdglue(consensus, networks[:n], test_subset, test_labels, memory, model)
        
        print(f"{n} network(s): {acc*100:.2f}%")

print("\n➡️ Performance generally improves with more diverse networks!")

## Key Takeaways

1. **Neuro-Symbolic Fusion**: HD-Glue creates a symbolic VSA layer over neural networks
   - Encodes neural embeddings as hypervectors
   - Preserves network knowledge in symbolic form

2. **Consensus Learning**: Multiple networks vote on predictions
   - Diverse networks provide better consensus
   - Can outperform individual networks

3. **Architecture-Agnostic**: Works with any neural network
   - CNNs, ResNets, Transformers, MLPs
   - Different architectures can be fused together

4. **Online Learning**: Dynamic and adaptive
   - Add new networks without retraining
   - Error correction via additional models
   - Remove underperforming models

5. **Efficient**: VSA operations are fast
   - Bit operations for binary VSA
   - No gradient descent needed
   - Minimal overhead compared to NNs

6. **Interpretable**: Symbolic representations
   - Can inspect hypervectors
   - Understand which networks contribute

## Next Steps

**Extend this tutorial**:
- Use real CNNs (ResNet, VGG) instead of MLPs
- Test on CIFAR-10 or CIFAR-100
- Fuse networks from different modalities (vision + audio)
- Implement weighted consensus (better networks get more weight)
- Life-long learning: accumulate models over time

**Related tutorials**:
- [Tutorial 2: Knowledge Graph Reasoning](02_knowledge_graph.md) - Symbolic reasoning
- [Tutorial 7: Hierarchical Structures](07_hierarchical_structures.md) - Compositional encoding
- [Tutorial 8: Multi-Modal Grounding](08_multimodal_grounding.md) - Heterogeneous fusion

## References

- Sutor et al. (2022). "Gluing Neural Networks Symbolically Through Hyperdimensional Computing." IJCNN.
- Kanerva (2009). "Hyperdimensional Computing."
- Mitrokhin et al. (2020). "Symbolic representation and learning with hyperdimensional computing."

## Running This Tutorial

**Requirements**:
```bash
pip install vsax scikit-learn matplotlib
```

**Run notebook**:
```bash
jupyter notebook examples/notebooks/tutorial_09_neural_symbolic_fusion.ipynb
```

**Note**: This tutorial uses sklearn's MLP for simplicity. For production, use JAX/Flax or PyTorch CNNs and extract embeddings from intermediate layers.